In [1]:
from ib_insync import *
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [2]:
ib = IB()
ib.connect()
ib.positions()

[Position(account='DU9096495', contract=CFD(conId=143916318, symbol='EUR', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), position=-1000.0, avgCost=1.0836)]

In [5]:
dt.datetime.utcnow().time()

datetime.time(16, 11, 9, 590663)

In [6]:
# strategy params
freq = "1 min"
window = 1
units = 1000
end_time = dt.time(16, 15, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

## SMA Day Trading Strategy

In [7]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # get current position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # identify required trades
    trades = target - current_pos
        
    # trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)
  

In [8]:
# start trading session
session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc")
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if dt.datetime.utcnow().time() >= end_time:
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

,close,returns,position
date,,,
2024-04-08 21:15:00+00:00,1.085820,NaN,NaN
2024-04-08 21:16:00+00:00,1.085820,0.000000,-0.0
2024-04-08 21:17:00+00:00,1.085815,-0.000005,1.0
2024-04-08 21:18:00+00:00,1.085830,0.000014,-1.0
2024-04-08 21:19:00+00:00,1.085815,-0.000014,1.0
...,...,...,...
2024-04-09 16:11:00+00:00,1.085775,0.000009,-1.0
2024-04-09 16:12:00+00:00,1.085830,0.000051,-1.0
2024-04-09 16:13:00+00:00,1.085890,0.000055,-1.0


,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,
2024-04-09 16:15:01+00:00,BOT,1000.0,1.0859,-4.3,-4.3


Session Stopped.
